#### Learned:
- .loc[]
- .idxmax()
- .where(df['column'] > 0, 0)
- df['column'] = pd.to_datetime(df['column'])
- df['new_column'] = df['column'].dt.day_name()
- df['column'].value_counts()

# Assignment 3 - Pandas DataFrame
## Part 1 (75 Points evenly distributed to first three questions)
The following code loads the olympics dataset (olympics.csv), which was derived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [1]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


In [21]:
df.loc['Afghanistan']

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1 

What is the first country in df?

*This function should return a Series.*

In [16]:
# You should write your whole answer within the function provided. 
def answer_zero():
    # This function should return the row for the first country, which is a Series object.
    return df.loc[list(df.index)[0]]

# You can examine what your function returns by calling it in the cell. 
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 2
In summer games, which nation has won the most gold medals?


*This function should return a single string value.*


In [22]:
def answer_one():
    return df.Gold.idxmax()

answer_one()

'United States'

### Question 3
Which nation had the biggest difference on gold medal counts? (between their summer and winter) 

*This function should return a single string value.*

In [31]:
def answer_one():
    df_c = df.copy()
    df_c['diff'] = abs(df_c['Gold'] - df_c['Gold.1'])
    return df_c['diff'].idxmax()

answer_one()

'United States'

# PART 2
## Question 4 (25 Points for first two questions + 25 Points Bonus for last two questions)

We will look at the publicly available airline data in this question similar to flights.csv that we covered in class practices. However, in this assignment, you are given 6 months of seperate data along with lookup tables for carriers and airports. Please apply data exploration and pre-procesing techniques and provide your answers for the following questions.



Questions: 

    1. What carrier has flown the 1st most number of flights? How many?  
    2. Which airport has the 3rd most delays? 
    3. What is the most popular day of the week to travel?
    4. What is the 1st and 5th most flown route? 

Hints:

1- pd.concat(list)
list=[A,B,C...] e.g. A= pd.read_csv("1.csv",encoding='utf-8')

2-please leverage from pandas dataframe features including 
groupby(...).size()...
groupby(...).sum().sort_values(....)

3- Dont forget to consider cancelled flights

4- Try to create a new column for "route"

Dataset Details: 
Dataset name: On-Time Performance,
Lookup Table: Carrier Lookup,
Lookup Table: Airport Lookup


In [39]:
pd.set_option('display.max_columns', None)

In [33]:
ID = pd.read_csv("L_AIRPORT_ID.csv",encoding='utf-8')

ID

,Code,Description
0,10001,"Afognak Lake, AK: Afognak Lake Airport"
1,10003,"Granite Mountain, AK: Bear Creek Mining Strip"
2,10004,"Lik, AK: Lik Mining Camp"
3,10005,"Little Squaw, AK: Little Squaw Airport"
4,10006,"Kizhuyak, AK: Kizhuyak Bay"
...,...,...
6356,16695,"Lafayette, TN: Lafayette Municipal"
6357,16696,"Smith Bay, AK: Smith Bay Airport"
6358,16697,"Moneta, VA: Smith Mountain Lake"
6359,16698,"Dune Lake, AK: Dune Lake Airport"


In [34]:
HISTORY = pd.read_csv("L_CARRIER_HISTORY.csv",encoding='utf-8')

HISTORY

,Code,Description
0,02Q,Titan Airways (2006 - )
1,04Q,Tradewind Aviation (2006 - )
2,05Q,"Comlux Aviation, AG (2006 - 2012)"
3,06Q,Master Top Linhas Aereas Ltd. (2007 - )
4,07Q,Flair Airlines Ltd. (2007 - )
...,...,...
1841,ZW,Air Wisconsin Airlines Corp (1994 - )
1842,ZX,Airbc Ltd. (1990 - 2000)
1843,ZX,Air Georgian (2002 - )
1844,ZY,Atlantic Gulf Airlines (1985 - 1986)


In [40]:
A = pd.read_csv("1.csv",encoding='utf-8')

A

,FL_DATE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,Unnamed: 50
0,2014-01-01,AA,19805,AA,N002AA,2377,12278,1227802,30928,ICT,"Wichita, KS",KS,20,Kansas,62,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,1135,1144.0,9.0,9.0,0.0,0.0,1100-1159,12.0,1156.0,1253.0,9.0,1300,1302.0,2.0,2.0,0.0,0.0,1300-1359,0.0,0.0,85.0,78.0,57.0,1.0,328.0,2,NaN
1,2014-01-01,AA,19805,AA,N3FXAA,2378,13303,1330303,32467,MIA,"Miami, FL",FL,12,Florida,33,15304,1530402,33195,TPA,"Tampa, FL",FL,12,Florida,33,2225,2220.0,-5.0,0.0,0.0,-1.0,2200-2259,14.0,2234.0,2311.0,4.0,2325,2315.0,-10.0,0.0,0.0,-1.0,2300-2359,0.0,0.0,60.0,55.0,37.0,1.0,204.0,1,NaN
2,2014-01-01,EV,20366,EV,N906EV,2500,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,12191,1219102,31453,HOU,"Houston, TX",TX,48,Texas,74,2105,NaN,NaN,NaN,NaN,NaN,2100-2159,NaN,NaN,NaN,NaN,2205,NaN,NaN,NaN,NaN,NaN,2200-2259,1.0,0.0,60.0,NaN,NaN,1.0,247.0,1,NaN
3,2014-01-01,EV,20366,EV,N903EV,2502,11146,1114603,31146,CRW,"Charleston/Dunbar, WV",WV,54,West Virginia,39,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,1655,1805.0,70.0,70.0,1.0,4.0,1600-1659,6.0,1811.0,1941.0,24.0,1900,2005.0,65.0,65.0,1.0,4.0,1900-1959,0.0,0.0,185.0,180.0,150.0,1.0,946.0,4,NaN
4,2014-01-01,EV,20366,EV,N903EV,2502,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,11146,1114603,31146,CRW,"Charleston/Dunbar, WV",WV,54,West Virginia,39,1320,1440.0,80.0,80.0,1.0,5.0,1300-1359,9.0,1449.0,1730.0,5.0,1625,1735.0,70.0,70.0,1.0,4.0,1600-1659,0.0,0.0,125.0,115.0,101.0,1.0,946.0,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471944,2014-01-31,OO,20304,OO,N292SW,5313,12889,1288903,32211,LAS,"Las Vegas, NV",NV,32,Nevada,85,11638,1163805,31638,FAT,"Fresno, CA",CA,6,California,91,910,905.0,-5.0,0.0,0.0,-1.0,0900-0959,11.0,916.0,1028.0,3.0,1038,1031.0,-7.0,0.0,0.0,-1.0,1000-1059,0.0,0.0,88.0,86.0,72.0,1.0,259.0,2,NaN
471945,2014-01-31,OO,20304,OO,N580SW,5314,12892,1289203,32575,LAX,"Los Angeles, CA",CA,6,California,91,14689,1468902,34689,SBA,"Santa Barbara, CA",CA,6,California,91,929,924.0,-5.0,0.0,0.0,-1.0,0900-0959,15.0,939.0,1000.0,4.0,1016,1004.0,-12.0,0.0,0.0,-1.0,1000-1059,0.0,0.0,47.0,40.0,21.0,1.0,89.0,1,NaN
471946,2014-01-31,OO,20304,OO,N580SW,5314,14689,1468902,34689,SBA,"Santa Barbara, CA",CA,6,California,91,12892,1289203,32575,LAX,"Los Angeles, CA",CA,6,California,91,1041,1039.0,-2.0,0.0,0.0,-1.0,1000-1059,24.0,1103.0,1132.0,11.0,1132,1143.0,11.0,11.0,0.0,0.0,1100-1159,0.0,0.0,51.0,64.0,29.0,1.0,89.0,1,NaN
471947,2014-01-31,OO,20304,OO,N216SW,5315,11292,1129202,30325,DEN,"Denver, CO",CO,8,Colorado,82,14543,1454302,34543,RKS,"Rock Springs, WY",WY,56,Wyoming,88,845,928.0,43.0,43.0,1.0,2.0,0800-0859,38.0,1006.0,1106.0,4.0,1014,1110.0,56.0,56.0,1.0,3.0,1000-1059,0.0,0.0,89.0,102.0,60.0,1.0,260.0,2,NaN


In [41]:
B = pd.read_csv("2.csv",encoding='utf-8')

B

,FL_DATE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,Unnamed: 50
0,2014-02-01,EV,20366,EV,N908EV,2537,12191,1219102,31453,HOU,"Houston, TX",TX,48,Texas,74,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,625,618.0,-7.0,0.0,0.0,-1.0,0600-0659,8.0,626.0,705.0,7.0,730,712.0,-18.0,0.0,0.0,-2.0,0700-0759,0.0,0.0,65.0,54.0,39.0,1.0,247.0,1,NaN
1,2014-02-01,EV,20366,EV,N904EV,2538,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,14814,1481402,30476,SHV,"Shreveport, LA",LA,22,Louisiana,72,845,840.0,-5.0,0.0,0.0,-1.0,0800-0859,16.0,856.0,926.0,4.0,940,930.0,-10.0,0.0,0.0,-1.0,0900-0959,0.0,0.0,55.0,50.0,30.0,1.0,190.0,1,NaN
2,2014-02-01,EV,20366,EV,N904EV,2538,14814,1481402,30476,SHV,"Shreveport, LA",LA,22,Louisiana,72,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,1010,1002.0,-8.0,0.0,0.0,-1.0,1000-1059,7.0,1009.0,1048.0,16.0,1105,1104.0,-1.0,0.0,0.0,-1.0,1100-1159,0.0,0.0,55.0,62.0,39.0,1.0,190.0,1,NaN
3,2014-02-01,EV,20366,EV,N876AS,2539,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,10747,1074702,30747,BRO,"Brownsville, TX",TX,48,Texas,74,1845,1839.0,-6.0,0.0,0.0,-1.0,1800-1859,15.0,1854.0,2013.0,4.0,2015,2017.0,2.0,2.0,0.0,0.0,2000-2059,0.0,0.0,90.0,98.0,79.0,1.0,482.0,2,NaN
4,2014-02-01,EV,20366,EV,N18557,3810,12266,1226603,31453,IAH,"Houston, TX",TX,48,Texas,74,14683,1468303,33214,SAT,"San Antonio, TX",TX,48,Texas,74,1152,1149.0,-3.0,0.0,0.0,-1.0,1100-1159,11.0,1200.0,1241.0,2.0,1258,1243.0,-15.0,0.0,0.0,-1.0,1200-1259,0.0,0.0,66.0,54.0,41.0,1.0,191.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430597,2014-02-28,AA,19805,AA,N3KKAA,1599,13303,1330303,32467,MIA,"Miami, FL",FL,12,Florida,33,12953,1295302,31703,LGA,"New York, NY",NY,36,New York,22,1545,1543.0,-2.0,0.0,0.0,-1.0,1500-1559,11.0,1554.0,1815.0,5.0,1840,1820.0,-20.0,0.0,0.0,-2.0,1800-1859,0.0,0.0,175.0,157.0,141.0,1.0,1096.0,5,NaN
430598,2014-02-28,AA,19805,AA,N493AA,1600,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,13851,1385103,33851,OKC,"Oklahoma City, OK",OK,40,Oklahoma,73,1125,1127.0,2.0,2.0,0.0,0.0,1100-1159,12.0,1139.0,1217.0,4.0,1220,1221.0,1.0,1.0,0.0,0.0,1200-1259,0.0,0.0,55.0,54.0,38.0,1.0,175.0,1,NaN
430599,2014-02-28,AA,19805,AA,N493AA,1600,13851,1385103,33851,OKC,"Oklahoma City, OK",OK,40,Oklahoma,73,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,1300,1259.0,-1.0,0.0,0.0,-1.0,1300-1359,8.0,1307.0,1335.0,10.0,1405,1345.0,-20.0,0.0,0.0,-2.0,1400-1459,0.0,0.0,65.0,46.0,28.0,1.0,175.0,1,NaN
430600,2014-02-28,AA,19805,AA,N437AA,1602,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,14893,1489302,33192,SMF,"Sacramento, CA",CA,6,California,91,945,943.0,-2.0,0.0,0.0,-1.0,0900-0959,13.0,956.0,1132.0,7.0,1135,1139.0,4.0,4.0,0.0,0.0,1100-1159,0.0,0.0,230.0,236.0,216.0,1.0,1431.0,6,NaN


In [37]:
C = pd.read_csv("3.csv",encoding='utf-8')

D = pd.read_csv("4.csv",encoding='utf-8')

E = pd.read_csv("5.csv",encoding='utf-8')

F = pd.read_csv("6.csv",encoding='utf-8')


In [38]:
print(ID.shape, HISTORY.shape, A.shape, B.shape, C.shape, D.shape, E.shape, F.shape)

(6361, 2) (1846, 2) (471949, 51) (430602, 51) (503758, 51) (483499, 51) (499278, 51) (502617, 51)


In [44]:
list_of_months = [A,B,C,D,E,F]
flight = pd.concat(list_of_months)

In [45]:
flight.shape

(2891703, 51)

### 1. What carrier has flown the 1st most number of flights? How many?  

In [47]:
flight.CARRIER.value_counts()

WN    579338
DL    383526
EV    350442
OO    304989
AA    267766
UA    242908
US    207776
MQ    202504
B6    123566
AS     76734
FL     49774
F9     37794
HA     36042
VX     28544
Name: CARRIER, dtype: int64

In [52]:
HISTORY.loc[HISTORY['Code'] == 'WN']

,Code,Description
1712,WN,Southwest Airlines Co. (1979 - )


#### Answer: Southwest Airlines Co. has flown the 1st most number of flights: 579338

### 2. Which airport has the 3rd most delays? 


#### Assumptions
1. Flights that came in early should not reduce the total counts of delay
2. I use delay group instead of actual delay time because I think it's a good representation of delay situation

In [66]:
# First, we create extra columns to filter out flights that come early
# Because when we sum up the delay time, these negative values would decrease the total delay time
# (flights that came in early should not reduce the total counts of delay)
# This would lead to an inaccurate estimate of total delay time

flight['departure_delay_without_negative'] = flight['DEP_DELAY_GROUP'].where(flight['DEP_DELAY_GROUP'] > 0, 0)
flight['arrival_delay_without_negative'] = flight['ARR_DELAY_GROUP'].where(flight['ARR_DELAY_GROUP'] > 0, 0)
flight

,FL_DATE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,Unnamed: 50,departure_delay_without_negative,arrival_delay_without_negative
0,2014-01-01,AA,19805,AA,N002AA,2377,12278,1227802,30928,ICT,"Wichita, KS",KS,20,Kansas,62,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,1135,1144.0,9.0,9.0,0.0,0.0,1100-1159,12.0,1156.0,1253.0,9.0,1300,1302.0,2.0,2.0,0.0,0.0,1300-1359,0.0,0.0,85.0,78.0,57.0,1.0,328.0,2,NaN,0.0,0.0
1,2014-01-01,AA,19805,AA,N3FXAA,2378,13303,1330303,32467,MIA,"Miami, FL",FL,12,Florida,33,15304,1530402,33195,TPA,"Tampa, FL",FL,12,Florida,33,2225,2220.0,-5.0,0.0,0.0,-1.0,2200-2259,14.0,2234.0,2311.0,4.0,2325,2315.0,-10.0,0.0,0.0,-1.0,2300-2359,0.0,0.0,60.0,55.0,37.0,1.0,204.0,1,NaN,0.0,0.0
2,2014-01-01,EV,20366,EV,N906EV,2500,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,12191,1219102,31453,HOU,"Houston, TX",TX,48,Texas,74,2105,NaN,NaN,NaN,NaN,NaN,2100-2159,NaN,NaN,NaN,NaN,2205,NaN,NaN,NaN,NaN,NaN,2200-2259,1.0,0.0,60.0,NaN,NaN,1.0,247.0,1,NaN,0.0,0.0
3,2014-01-01,EV,20366,EV,N903EV,2502,11146,1114603,31146,CRW,"Charleston/Dunbar, WV",WV,54,West Virginia,39,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,1655,1805.0,70.0,70.0,1.0,4.0,1600-1659,6.0,1811.0,1941.0,24.0,1900,2005.0,65.0,65.0,1.0,4.0,1900-1959,0.0,0.0,185.0,180.0,150.0,1.0,946.0,4,NaN,4.0,4.0
4,2014-01-01,EV,20366,EV,N903EV,2502,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,11146,1114603,31146,CRW,"Charleston/Dunbar, WV",WV,54,West Virginia,39,1320,1440.0,80.0,80.0,1.0,5.0,1300-1359,9.0,1449.0,1730.0,5.0,1625,1735.0,70.0,70.0,1.0,4.0,1600-1659,0.0,0.0,125.0,115.0,101.0,1.0,946.0,4,NaN,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502612,2014-06-30,AA,19805,AA,N5DVAA,2496,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,12892,1289203,32575,LAX,"Los Angeles, CA",CA,6,California,91,1615,1610.0,-5.0,0.0,0.0,-1.0,1600-1659,10.0,1620.0,1657.0,36.0,1720,1733.0,13.0,13.0,0.0,0.0,1700-1759,0.0,0.0,185.0,203.0,157.0,1.0,1235.0,5,NaN,0.0,0.0
502613,2014-06-30,AA,19805,AA,N5CPAA,2497,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,12892,1289203,32575,LAX,"Los Angeles, CA",CA,6,California,91,2235,2303.0,28.0,28.0,1.0,1.0,2200-2259,9.0,2312.0,2346.0,3.0,2340,2349.0,9.0,9.0,0.0,0.0,2300-2359,0.0,0.0,185.0,166.0,154.0,1.0,1235.0,5,NaN,1.0,0.0
502614,2014-06-30,AA,19805,AA,N202AA,2498,11292,1129202,30325,DEN,"Denver, CO",CO,8,Colorado,82,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,905,904.0,-1.0,0.0,0.0,-1.0,0900-0959,15.0,919.0,1149.0,9.0,1200,1158.0,-2.0,0.0,0.0,-1.0,1200-1259,0.0,0.0,115.0,114.0,90.0,1.0,641.0,3,NaN,0.0,0.0
502615,2014-06-30,AA,19805,AA,N202AA,2498,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,11292,1129202,30325,DEN,"Denver, CO",CO,8,Colorado,82,730,722.0,-8.0,0.0,0.0,-1.0,0700-0759,13.0,735.0,810.0,13.0,825,823.0,-2.0,0.0,0.0,-1.0,0800-0859,0.0,0.0,115.0,121.0,95.0,1.0,641.0,3,NaN,0.0,0.0


In [74]:
# Departure delay 

departure_delay = flight.groupby('ORIGIN_AIRPORT_ID')['departure_delay_without_negative'].sum()
departure_delay

ORIGIN_AIRPORT_ID
10135     557.0
10136     899.0
10140    8614.0
10141     101.0
10146     250.0
          ...  
15841     185.0
15897      23.0
15919    4812.0
15991     144.0
16218     445.0
Name: departure_delay_without_negative, Length: 319, dtype: float64

In [75]:
# Arrival delay

arrival_delay = flight.groupby('DEST_AIRPORT_ID')['arrival_delay_without_negative'].sum()
arrival_delay

DEST_AIRPORT_ID
10135     544.0
10136     911.0
10140    9729.0
10141     149.0
10146     300.0
          ...  
15841     174.0
15897       4.0
15919    4943.0
15991      89.0
16218     433.0
Name: arrival_delay_without_negative, Length: 319, dtype: float64

In [85]:
# Merged above two dataframes

total_delay = pd.concat([departure_delay, arrival_delay], axis = 1)
total_delay['total_delay'] = total_delay['departure_delay_without_negative'] + total_delay['arrival_delay_without_negative']
total_delay.sort_values(by = 'total_delay', ascending = False).head(3)

,departure_delay_without_negative,arrival_delay_without_negative,total_delay
13930,142777.0,141275.0,284052.0
10397,118276.0,114999.0,233275.0
11298,94073.0,93863.0,187936.0


In [158]:
ID.loc[ID['Code'] == 11298]['Description'].values[0]

'Dallas/Fort Worth, TX: Dallas/Fort Worth International'

#### Answer: Dallas/Fort Worth, TX: Dallas/Fort Worth Interational Airport has the 3rd most delays

With 2 assumptions:
- Flights that came in early should not reduce the total counts of delay
- I use delay group instead of actual delay time because I think it's a good representation of delay situation

### 3. What is the most popular day of the week to travel?

In [109]:
flight['FL_DATE'] = pd.to_datetime(flight['FL_DATE'])
flight['day_of_the_week'] = flight['FL_DATE'].dt.day_name()

flight['day_of_the_week'].value_counts()

Friday       437210
Thursday     436422
Monday       433913
Wednesday    424036
Sunday       405367
Tuesday      405063
Saturday     349692
Name: day_of_the_week, dtype: int64

#### Answer: Friday is the most popular day of the week to travel

### 4. What is the 1st and 5th most flown route? 

In [120]:
flight['route'] = flight['ORIGIN_AIRPORT_ID']* 1000000 + flight['DEST_AIRPORT_ID']
flight['route']

0         12278011298
1         13303015304
2         11298012191
3         11146011298
4         11298011146
             ...     
502612    11298012892
502613    11298012892
502614    11292011298
502615    11298011292
502616    11298012892
Name: route, Length: 2891703, dtype: int64

In [126]:
result = list(flight['route'].value_counts().head(5).index)
result

[14771012892, 12892014771, 12889012892, 12892012889, 12478012892]

In [137]:
str(result[0])[0:5]

'14771'

In [138]:
# arrival airport ID 
str(result[0])[-5:]

'12892'

In [177]:
def parse_result(list):
    list_result = []
    for x in list:
        origin_code = str(x)[0:5] # departure airport ID
        destination_code = str(x)[-5:] # arrival airport ID 
        
        origin = ID.loc[ID['Code'] == int(origin_code)]['Description'].values[0]
        destination = ID.loc[ID['Code'] == int(destination_code)]['Description'].values[0]
                
        list_result.append((origin, destination))
    
    return(list_result)

#### Answer:

In [187]:
answer = parse_result(result)

for x in range(len(answer)):
    print('The number ' + str(x+1) + ' most flown route is from: \n' + answer[x][0] + '\n to: \n' + answer[x][1] + '\n\n')

The number 1 most flown route is from: 
San Francisco, CA: San Francisco International
 to: 
Los Angeles, CA: Los Angeles International


The number 2 most flown route is from: 
Los Angeles, CA: Los Angeles International
 to: 
San Francisco, CA: San Francisco International


The number 3 most flown route is from: 
Las Vegas, NV: McCarran International
 to: 
Los Angeles, CA: Los Angeles International


The number 4 most flown route is from: 
Los Angeles, CA: Los Angeles International
 to: 
Las Vegas, NV: McCarran International


The number 5 most flown route is from: 
New York, NY: John F. Kennedy International
 to: 
Los Angeles, CA: Los Angeles International


